In [1]:
!pip install pygame

In [1]:
import pygame
import random
import numpy as np
import math

#Konstante
REDOVI = 6
STUPCI = 7
VISINA = 600
SIRINA = 700
CHELIJA = 100
HI = 1
AI = 2
DUBINA = 4

#Boje
ZUTA = (230, 191, 0)
CRVENA = (230, 0, 0)
PLAVA = (0, 0, 179)
CRNA = (0, 0, 0)

#Funkcija crtanja ploče i žetona
def plocha_nacrtaj():
    for red in range(REDOVI):
        for stup in range(STUPCI):
            pygame.draw.rect(prozor, PLAVA, (stup * CHELIJA, red * CHELIJA, CHELIJA, CHELIJA))
            pygame.draw.line(prozor, CRNA, (stup * CHELIJA, 0), (stup * CHELIJA, VISINA), 2)
            pygame.draw.line(prozor, CRNA, (0, red * CHELIJA), (SIRINA, red * CHELIJA), 2)
            if plocha[red, stup] == HI:
                pygame.draw.circle(prozor, ZUTA, (stup * CHELIJA + 50, red * CHELIJA + 50), 40)
            elif plocha[red, stup] == AI:
                pygame.draw.circle(prozor, CRVENA, (stup * CHELIJA + 50, red * CHELIJA + 50), 40)

#Funkcija koja provjerava pobjedu
def pobjeda(zeton):
    #Horizontalna provjera
    for red in range(REDOVI):
        for stup in range(STUPCI - 3):
            if plocha[red, stup] == zeton and plocha[red, stup + 1] == zeton and \
               plocha[red, stup + 2] == zeton and plocha[red, stup + 3] == zeton:
                return True
    #Vertikalna provjera
    for red in range(REDOVI - 3):
        for stup in range(STUPCI):
            if plocha[red, stup] == zeton and plocha[red + 1, stup] == zeton and \
               plocha[red + 2, stup] == zeton and plocha[red + 3, stup] == zeton:
                return True
    #Dijagonalna provjera (gore-lijevo -> dolje-desno)
    for red in range(REDOVI - 3):
        for stup in range(STUPCI - 3):
            if plocha[red, stup] == zeton and plocha[red + 1, stup + 1] == zeton and \
               plocha[red + 2, stup + 2] == zeton and plocha[red + 3, stup + 3] == zeton:
                return True
    #Dijagonalna provjera (dolje-lijevo -> gore-desno)
    for red in range(3, REDOVI):
        for stup in range(STUPCI - 3):
            if plocha[red, stup] == zeton and plocha[red - 1, stup + 1] == zeton and \
               plocha[red - 2, stup + 2] == zeton and plocha[red - 3, stup + 3] == zeton:
                return True
    return False

#Funkcija za ispis poruke pobjede
def pobjeda_poruka(zeton):
    font = pygame.font.SysFont("Arialblack", 30)
    tekst = font.render(f"Igrač {zeton} je pobjedio!", True, ZUTA if zeton == HI else CRVENA)
    tekst_oblik = tekst.get_rect(center=(width // 2, 20))
    pygame.draw.rect(prozor, CRNA, (tekst_oblik.x - 5, tekst_oblik.y - 5, tekst_oblik.width + 10, tekst_oblik.height + 10))
    prozor.blit(tekst, tekst_oblik)
    pygame.display.update()
    pygame.time.delay(2000)
    
#Funkcija za ispis poruke izjednacenja
def izjednacenje_poruka():
    font = pygame.font.SysFont("Arialblack", 30)
    tekst = font.render(f"Izjednačenje!", True, PLAVA)
    tekst_oblik = tekst.get_rect(center=(width // 2, 20))
    pygame.draw.rect(prozor, CRNA, (tekst_oblik.x - 5, tekst_oblik.y - 5, tekst_oblik.width + 10, tekst_oblik.height + 10))
    prozor.blit(tekst, tekst_oblik)
    pygame.display.update()
    pygame.time.delay(2000)

#Funkcija za AI potez i pozivanje algoritma
def ai_potez():
    dostupno = [stup for stup in range(STUPCI) if plocha[0, stup] == 0]
    if dostupno:
        m_stup = -1
        m_bodovi = -math.inf
        for stup in dostupno:
            red = prazan_red(stup)
            plocha[red, stup] = AI
            bodovi = minimax(plocha, DUBINA, -math.inf, math.inf, False)
            plocha[red, stup] = 0
            if bodovi > m_bodovi:
                m_bodovi = bodovi
                m_stup = stup
        return m_stup
    return None

#Funkcija koja traži prvi prazan red
def prazan_red(stup):
    for red in range(REDOVI - 1, -1, -1):
        if plocha[red, stup] == 0:
            return red
    return None

#Funkcija minimax algoritma sa alfa-beta orezivanjem
def minimax(plocha, dubina, alfa, beta, maksimiziranje):
    if dubina == 0 or pobjeda(HI) or pobjeda(AI):
        return pozicija_bodovanje(plocha, AI)
    if maksimiziranje:
        maksimum = -math.inf
        for stup in range(STUPCI):
            if plocha[0, stup] == 0:
                red = prazan_red(stup)
                plocha[red, stup] = AI
                eval = minimax(plocha, dubina - 1, alfa, beta, False)
                plocha[red, stup] = 0
                maksimum = max(maksimum, eval)
                alfa = max(alfa, eval)
                if beta <= alfa:
                    break
        return maksimum
    else:
        minimum = math.inf
        for stup in range(STUPCI):
            if plocha[0, stup] == 0:
                red = prazan_red(stup)
                plocha[red, stup] = HI
                eval = minimax(plocha, dubina - 1, alfa, beta, True)
                plocha[red, stup] = 0
                minimum = min(minimum, eval)
                beta = min(beta, eval)
                if beta <= alfa:
                    break
        return minimum

#Funkcije za dodavanje bodova
def pozicija_bodovanje(plocha, zeton):
    bodovi = 0
    #Bodovanje centra
    centar = [int(i) for i in list(plocha[:, STUPCI // 2])]
    bodovi += centar.count(zeton) * 5
    #Horizontalno bodovanje
    for red in range(REDOVI):
        for stup in range(STUPCI - 3):
            bodovi += plocha_bodovanje([plocha[red, stup + i] for i in range(4)], zeton)
    #Vertikalno bodovanje
    for red in range(REDOVI - 3):
        for stup in range(STUPCI):
            bodovi += plocha_bodovanje([plocha[red + i, stup] for i in range(4)], zeton)
    #Dijagonalno bodovanje (gore-lijevo -> dolje-desno)
    for red in range(REDOVI - 3):
        for stup in range(STUPCI - 3):
            bodovi += plocha_bodovanje([plocha[red + i, stup + i] for i in range(4)], zeton)
    #Dijagonalno bodovanje (dolje-lijevo -> gore-desno)
    for red in range(3, REDOVI):
        for stup in range(STUPCI - 3):
            bodovi += plocha_bodovanje([plocha[red - i, stup + i] for i in range(4)], zeton)       
    return bodovi

def plocha_bodovanje(plocha, zeton):
    bodovi = 0
    protivnik = HI if zeton == AI else AI
    if plocha.count(zeton) == 4:
        bodovi += 100
    elif plocha.count(zeton) == 3 and plocha.count(0) == 1:
        bodovi += 4
    elif plocha.count(zeton) == 2 and plocha.count(0) == 2:
        bodovi += 2
    if plocha.count(protivnik) == 3 and plocha.count(0) == 1:
        bodovi -= 6
    return bodovi

#Inicijaliziranje igre
plocha = np.zeros((REDOVI, STUPCI))
pygame.init()
width, height = SIRINA, VISINA
prozor = pygame.display.set_mode((width, height))
pygame.display.set_caption("Connect Four")

#Početak igre
pokrenuto = True
na_redu = HI
#na_redu = AI
#na_redu = random.choice([HI, AI])

plocha_nacrtaj()
pygame.display.update()
while pokrenuto:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pokrenuto = False
        elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            stup = event.pos[0] // CHELIJA
            if plocha[0, stup] == 0:
                red = prazan_red(stup)
                if red is not None:
                    plocha[red, stup] = na_redu
                    if pobjeda(na_redu):
                        plocha_nacrtaj()
                        pygame.display.update()
                        pobjeda_poruka(na_redu)
                        pokrenuto = False
                        break
                    elif np.all(plocha != 0):
                        plocha_nacrtaj()
                        pygame.display.update()
                        izjednacenje_poruka()
                        pokrenuto = False
                        break
                    plocha_nacrtaj()
                    pygame.display.update()
                    na_redu = 3 - na_redu
                    
    if na_redu == AI and pokrenuto:
        stup = ai_potez()
        if stup is not None:
            red = prazan_red(stup)
            if red is not None:
                plocha[red, stup] = na_redu
                if pobjeda(na_redu):
                    plocha_nacrtaj()
                    pygame.display.update()
                    pobjeda_poruka(na_redu)
                    pokrenuto = False
                    break
                elif np.all(plocha != 0):
                    plocha_nacrtaj()
                    pygame.display.update()
                    izjednacenje_poruka()
                    pokrenuto = False
                    break
                plocha_nacrtaj()
                pygame.display.update()
                na_redu = 3 - na_redu
pygame.quit()

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
